<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250701_GCP_Cloud_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 步驟1-1

#### 授予Colab權限

In [1]:
from google.colab import auth
auth.authenticate_user()

#### 使用CLI指令更改專案ID與地區

In [2]:
!gcloud config set project tibame-gad251-31-0701
!gcloud config set run/region asia-east1

Updated property [core/project].
Updated property [run/region].


#### 步驟 1-2

#### 撰寫應用程式

In [3]:
# 請參考你提供的完整程式碼，略過重複貼上
# 建議存檔方式：
main_py = '''
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from linebot.exceptions import InvalidSignatureError
from google.cloud import bigquery
import os

app = Flask(__name__)

# 讀取環境變數
LINE_CHANNEL_ACCESS_TOKEN = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = os.getenv('LINE_CHANNEL_SECRET')
PROJECT_ID = os.getenv('GCP_PROJECT_ID')
DATASET_ID = os.getenv('BQ_DATASET_ID')
MODEL_ID = os.getenv('BQ_MODEL_ID')

line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)
bq_client = bigquery.Client()

def parse_user_input(text):
    try:
        return dict(item.strip().split("=") for item in text.split(","))
    except Exception:
        return None

def predict_income(data):
    query = f"""
    WITH prediction AS (
      SELECT
        predicted_income_bracket,
        predicted_income_bracket_probs
      FROM
        ML.PREDICT (
          MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_ID}`,
          (
            SELECT
              {data['age']} AS age,
              '{data['workclass']}' AS workclass,
              '{data['marital_status']}' AS marital_status,
              {data['education_num']} AS education_num,
              '{data['occupation']}' AS occupation,
              {data['hours_per_week']} AS hours_per_week
              -- Removed income_bracket from input features
          )
        )
    )
    SELECT
      predicted_income_bracket,
      MAX(IF(TRIM(LOWER(probs.label)) = '>50k', probs.prob, NULL)) AS prob_gt_50k,
      MAX(IF(TRIM(LOWER(probs.label)) = '<=50k', probs.prob, NULL)) AS prob_le_50k
    FROM prediction, UNNEST(predicted_income_bracket_probs) AS probs
    GROUP BY predicted_income_bracket
    LIMIT 1
    """
    result = bq_client.query(query).result()
    row = next(result)
    return row.predicted_income_bracket, row.prob_gt_50k, row.prob_le_50k
@app.route("/")
def index():
    return "Hello from Cloud Run!"

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_input = event.message.text
    parsed = parse_user_input(user_input)

    if parsed is None:
        reply = """請輸入格式正確的資料，例如：
age=45, education_num=13, occupation=Exec-managerial, \
hours_per_week=50, workclass=Private, \
marital_status=Married-civ-spouse""" # Removed income_bracket from example input
    else:
        try:
            # Use a single multiline f-string for the reply
            label, prob_gt_50k, prob_le_50k = predict_income(parsed)

            # Handle potential None values for probabilities
            prob_gt_50k_str = f"{round(prob_gt_50k * 100, 2)}%" if prob_gt_50k is not None else "無法取得"
            prob_le_50k_str = f"{round(prob_le_50k * 100, 2)}%" if prob_le_50k is not None else "無法取得"

            reply = f"""預測結果：{label}
>50K 機率：{prob_gt_50k_str}
<=50K 機率：{prob_le_50k_str}"""
        except Exception as e:
            reply = f"發生錯誤：{str(e)}"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply)
    )

if __name__ == '__main__':
    port = int(os.environ.get("PORT", 8080))
    app.run(host='0.0.0.0', port=port)
'''
with open("main.py", "w") as f:
    f.write(main_py)


#### 步驟2

#### 撰寫requirements.txt

In [4]:
with open("requirements.txt", "w") as f:
    f.write(
        "flask\n"
        "line-bot-sdk\n"
        "google-cloud-bigquery\n"
        "pandas\n"
        "gunicorn\n"
    )

#### 步驟3

#### 撰寫 Dockerfile 容器設定檔

In [7]:
dockerfile = '''\
FROM python:3.10-slim
WORKDIR /app
COPY requirements.txt ./
RUN pip install --upgrade pip && \
    pip install --no-cache-dir -r requirements.txt
COPY . .
ENV PORT=8080
CMD gunicorn --bind 0.0.0.0:$PORT main:app
'''
with open("Dockerfile", "w") as f:
    f.write(dockerfile)
